In [1]:
import numpy as np
import pandas as pd

In [2]:
# cols = ['Postcode', 'Borough', 'Neighbourhood']
# df = pd.read_clipboard(header=None, names=cols)
# df.to_csv('toronto_postcodes.csv', index=False)

df = pd.read_csv('toronto_postcodes.csv')

In [3]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace = True)

In [5]:
def neighbourhood_adjustment(df):
    """
    This function creating a list for the 'Neighnourhood' column. 
    if the 'Postcode' area of dataframe(df) has multiple neighbourhoods,
    it creates a list of them and changes the 'Neighnourhood' column.
    returns changed dataframe
    """
    uniq_ix = set(df.index.values)
    for ix in uniq_ix:
        if type(df.loc[ix, 'Neighbourhood']) is pd.Series:
            neigh_list = df.loc[ix, 'Neighbourhood'].values.tolist()
            neigh = neigh_list[0]
            for n in neigh_list[1:]:
                neigh += ', ' + n 
            df.loc[ix, 'Neighbourhood'] = neigh            
    return df

In [6]:
df_postal_indexed = df.set_index('Postcode')
df_postal_indexed_with_duplicates = neighbourhood_adjustment(df_postal_indexed)
df_postal_indexed_with_duplicates.head(7)

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,NaN


In [7]:
df_postal_indexed_no_duplicates = df_postal_indexed_with_duplicates.drop_duplicates('Neighbourhood')
df_postal_indexed_no_duplicates.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,NaN


In [8]:
df_postal_indexed_no_duplicates['Neighbourhood'].fillna(df_postal_indexed_no_duplicates['Borough'], inplace = True)

C:\Users\mhboz\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [9]:
df_clean = df_postal_indexed_no_duplicates.reset_index()
df_clean.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df_clean.to_csv('toronto_postcodes_v2.csv', index = False)

In [12]:
df_clean.shape

(103, 3)